In [1]:
import fitting_functions
fitting_functions.__file__

'/opt/conda/lib/python3.8/site-packages/fitting_functions-0.0.0-py3.8.egg/fitting_functions/__init__.py'

In [2]:
!rm -r /opt/conda/lib/python3.8/site-packages/fitting_functions*

In [3]:
cd ..

/home/jovyan


In [4]:
!python  setup.py install


running install
running bdist_egg
running egg_info
writing fitting_functions.egg-info/PKG-INFO
writing dependency_links to fitting_functions.egg-info/dependency_links.txt
writing requirements to fitting_functions.egg-info/requires.txt
writing top-level names to fitting_functions.egg-info/top_level.txt
reading manifest file 'fitting_functions.egg-info/SOURCES.txt'
writing manifest file 'fitting_functions.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/fitting_functions
copying build/lib/fitting_functions/__init__.py -> build/bdist.linux-x86_64/egg/fitting_functions
copying build/lib/fitting_functions/lineshapes.py -> build/bdist.linux-x86_64/egg/fitting_functions
byte-compiling build/bdist.linux-x86_64/egg/fitting_functions/__init__.py to __init__.cpython-38.pyc
byte-compiling build/bdist.linux-x86_64/egg/fitting_functions/lineshapes.py to linesh

In [5]:
cd examples/

/home/jovyan/examples


In [6]:
import lmfit
import numpy as np
import copy
import matplotlib.pyplot as plt
from fitting_functions import paramagnon_integrated_I
from matplotlib.ticker import AutoMinorLocator

%matplotlib widget

In [7]:
E_, I_ = np.loadtxt('LSCO_30_LH_grazout.txt', unpack=True, skiprows=1)
E_ *= -1
choose = np.logical_and(E_>-.5, E_<2.5)
E = E_[choose]
I = I_[choose]
dd_onset = 1.

In [8]:
model = (lmfit.models.GaussianModel(prefix='el_') + lmfit.Model(paramagnon_integrated_I, prefix='mag_')
         + lmfit.models.PseudoVoigtModel(prefix='dd0_')
         + lmfit.models.PseudoVoigtModel(prefix='dd1_')
         + lmfit.models.PseudoVoigtModel(prefix='dd2_')
         + lmfit.models.ConstantModel())
params = model.make_params()

fwhm = 2*np.sqrt(2*np.log(2))
res = 0.13/fwhm
                    
params['el_center'].set(value=0, vary=False)
params['el_amplitude'].set(value=3, min=0)
params['el_sigma'].set(value=res, vary=False)

params['mag_center'].set(value=.25)
params['mag_sigma'].set(value=.08, min=0)
params['mag_amplitude'].set(value=.36, min=0)

params['mag_res'].set(value=res, vary=False)
params['mag_kBT'].set(value=8.617e-5*25, vary=False)

params['dd0_center'].set(value=1.6, min=1, max=3)
params['dd0_sigma'].set(value=0.1, min=0)
params['dd0_amplitude'].set(value=300)

params['dd1_center'].set(value=1.8, min=1, max=3)
params['dd1_sigma'].set(value=0.1, min=0)
params['dd1_amplitude'].set(value=300)

params['dd2_center'].set(value=2, min=1, max=3)
params['dd2_sigma'].set(value=0.1, min=0)
params['dd2_amplitude'].set(value=300)

params_dd = copy.deepcopy(params)

for key in params_dd.keys():
    if key[:2] in ['el', 'ma']:
        params_dd[key].set(vary=False)
    else:
        params_dd[key].set(vary=True)

# Fit dds and force leading edge accuracy  by artificial weighting
dd_region = np.logical_or(E<-.3, E>dd_onset)
weights = .1 + np.exp(-1*((E-1.1)/.3)**2)
params_dd['c'].set(value=I.min(), vary=False)       
result_dds = model.fit(I[dd_region], x=E[dd_region], params=params_dd,
                      weights=weights[dd_region])

#fig, ax = plt.subplots()
#result_dds.plot_fit(ax=ax, show_init=True)

# assign and fix values for dds 
for key in params.keys():
    if key[:2] == 'dd':
        params[key].set(value=result_dds.params[key].value, vary=False)

params['c'].set(value=I.min(), vary=False)       
result = model.fit(I, x=E, params=params)

fig, ax = plt.subplots()
result.plot_fit(ax=ax, show_init=True)
# 
# print(result.fit_report())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
fig, ax = plt.subplots()

x_fit = np.linspace(E.min(), E.max(), 1000)

components = result.eval_components(x=x_fit)
constant = components.pop('constant')
dd0 = components.pop('dd0_')
dd1 = components.pop('dd1_')
dd2 = components.pop('dd2_')

BG = constant + dd0 + dd1 + dd2

ax.plot(x_fit, BG, 'k:', label='BG')
for model_name, model_value in components.items():
    ax.plot(x_fit, model_value + BG, '-', label=model_name.strip('_'))

y_fit = result.eval(**result.best_values, x=x_fit)
ax.plot(x_fit, y_fit, color=[0.5]*3, label='fit', lw=3, alpha=0.5)
ax.plot(E, I, 'k.', label='data')

ax.set_xlabel('Energy loss (eV)')
ax.set_ylabel('I')
ax.legend()
ax.axis([-.4, dd_onset, 0, 400])

ax.xaxis.set_minor_locator(AutoMinorLocator(2))
ax.yaxis.set_minor_locator(AutoMinorLocator(2))

print(result.fit_report())

result.dump(open('fit_info.json','w'))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[[Model]]
    (((((Model(gaussian, prefix='el_') + Model(paramagnon_integrated_I, prefix='mag_')) + Model(pvoigt, prefix='dd0_')) + Model(pvoigt, prefix='dd1_')) + Model(pvoigt, prefix='dd2_')) + Model(constant))
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 94
    # data points      = 98
    # variables        = 4
    chi-square         = 10094.5880
    reduced chi-square = 107.389234
    Akaike info crit   = 462.209149
    Bayesian info crit = 472.549019
[[Variables]]
    el_amplitude:   8.71051675 +/- 0.86007694 (9.87%) (init = 3)
    el_center:      0 (fixed)
    el_sigma:       0.05520592 (fixed)
    mag_amplitude:  82.9330335 +/- 1.74611563 (2.11%) (init = 0.36)
    mag_center:     0.26670709 +/- 0.00307822 (1.15%) (init = 0.25)
    mag_sigma:      0.22780824 +/- 0.01214697 (5.33%) (init = 0.08)
    mag_res:        0.05520592 (fixed)
    mag_kBT:        0.00215425 (fixed)
    dd0_amplitude:  680.3434 (fixed)
    dd0_center:     1.801213 (fixed)
    

16377

In [10]:
ci = result.ci_report()
with open('ci_info.text','w') as f:
    f.write(ci)